# Import library

In [76]:
import sys
sys.path.append('src')
import enex_analysis as enex
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import dartwork_mpl as dm
import warnings
from matplotlib.gridspec import GridSpec
import matplotlib.ticker as ticker
warnings.filterwarnings("ignore")
dm.use_style('dmpl_light')
mpl.rcParams.update({
    "text.usetex": False,
    "mathtext.fontset": "custom",
    "mathtext.rm": "Roboto",
    "mathtext.it": "Roboto:italic",
    "mathtext.bf": "Roboto:bold",
    "mathtext.sf": "Roboto",
    "mathtext.tt": "DejaVu Sans Mono",
    "mathtext.fallback": "stix",
    "mathtext.default": "it",
})

# Data

In [68]:
T0_range = np.linspace(-20, 10, 100)
T_w_serv = 40

Data = {'external unit': {}, 'refrigerant': {}, 'heat exchanger': {}, 'mixing valve': {}, 'whole': {}, 'temp': {}}

# External unit
for symbol in ['X_eff', 'X_fan', 'X_a_ext_out', 'X_a_ext_in', 'X_c_ext', 'X_r_ext', 'dV_a_ext', 'Q_a_ext_out', 'Q_a_ext_in', 'Q_r_ext', 'E_fan']:
    Data['external unit'][symbol] = []

# Refrigerant
for symbol in ['X_eff', 'X_cmp', 'X_r_ext', 'X_c_r', 'X_r_HX', 'E_cmp', 'Q_r_ext', 'Q_r_HX']:
    Data['refrigerant'][symbol] = []

# Heat exchanger
for symbol in ['X_eff', 'X_w_HX', 'X_r_HX', 'X_c_HX', 'X_w_sup_HX','Q_w_HX', 'Q_r_HX', 'Q_w_sup_HX']:
    Data['heat exchanger'][symbol] = []

# Mixing valve
for symbol in ['X_eff', 'X_w_serv', 'X_c_mix', 'X_w_sup_mix', 'X_w_HX', 'Q_w_serv', 'Q_w_sup_mix', 'Q_w_HX']:
    Data['mixing valve'][symbol] = []

# Whole
for symbol in ['Ex_eff', 'En_eff', 'COP', 'PLR']:
    Data['whole'][symbol] = []

# Temp
for symbol in ['T_w_sup', 'T0', 'T_w_serv', 'T_w_HX', 'T_r_HX']:
    Data['temp'][symbol] = []

# np.zeros((len(T0_range), len(T_w_serv_range)))
for i, T0 in enumerate(T0_range): # row
    ASHPB = enex.HeatPumpBoiler_without_tank()
    ASHPB.dV_w_serv = 7.5
    ASHPB.Q_r_max = 25000
    
    ASHPB.T0          = T0
    ASHPB.T_w_serv    = T_w_serv
    ASHPB.T_w_sup     = 0.2 * T0 + 10
    ASHPB.T_a_ext_out = T0 - 5
    ASHPB.T_r_ext     = T0 - 10
    
    ASHPB.T_r_exch    = T_w_serv + 10
    ASHPB.T_w_exch    = T_w_serv + 5
    ASHPB.system_update()
    
    # 시스템 에러 체크
    if ASHPB.Q_r_ext < 0:
        print(f'Q_r_ext: {ASHPB.Q_r_ext:.2f}')
        print(f'T0: {T0}, T_w_sup: {enex.K2C(ASHPB.T_w_sup):.2f}')
        print(f'T_a_ext_out: {enex.K2C(ASHPB.T_a_ext_out):.2f}')
        print(f'T_r_ext: {enex.K2C(ASHPB.T_r_ext):.2f}')
        print(f'X_cmp: {ASHPB.E_cmp:.2f}')
        print(f'Q_r_exch: {ASHPB.Q_r_exch:.2f}')
        print(f'Q_r_ext: {ASHPB.Q_r_ext:.2f}')
        print(f'COP: {ASHPB.COP:.2f}')
        print(f'PLR: {ASHPB.Q_r_exch / ASHPB.Q_r_max * 100:.2f}')
    if ASHPB.COP < 1:
        print(f'COP: {ASHPB.COP:.2f}')

    # Temp
    Data['temp']['T_w_sup'].append(enex.K2C(ASHPB.T_w_sup))
    Data['temp']['T0'].append(enex.K2C(ASHPB.T0))
    Data['temp']['T_w_serv'].append(enex.K2C(ASHPB.T_w_serv))
    Data['temp']['T_w_HX'].append(enex.K2C(ASHPB.T_w_exch))
    Data['temp']['T_r_HX'].append(enex.K2C(ASHPB.T_r_exch))

    # External unit exergy efficiency: X_a_ext_out / (E_fan + X_r_ext + X_a_ext_in)
    Data['external unit']['X_eff'].append((ASHPB.X_a_ext_out / (ASHPB.E_fan + ASHPB.X_r_ext + ASHPB.X_a_ext_in)) * 100)
    Data['external unit']['X_r_ext'].append(ASHPB.X_r_ext)
    Data['external unit']['X_fan'].append(ASHPB.X_fan)
    Data['external unit']['X_c_ext'].append(ASHPB.X_c_ext)
    Data['external unit']['X_a_ext_out'].append(ASHPB.X_a_ext_out)
    Data['external unit']['X_a_ext_in'].append(ASHPB.X_a_ext_in)
    Data['external unit']['dV_a_ext'].append(ASHPB.dV_a_ext)
    Data['external unit']['Q_a_ext_out'].append(ASHPB.Q_a_ext_out)
    Data['external unit']['Q_a_ext_in'].append(ASHPB.Q_a_ext_in)
    Data['external unit']['Q_r_ext'].append(ASHPB.Q_r_ext)
    Data['external unit']['E_fan'].append(ASHPB.E_fan)
    

    # Refrigerant loop exergy efficiency: (X_r_exch + X_r_ext) / E_cmp
    Data['refrigerant']['X_eff'].append((ASHPB.X_r_exch + ASHPB.X_r_ext) / ASHPB.E_cmp * 100)
    Data['refrigerant']['X_cmp'].append(ASHPB.X_cmp)
    Data['refrigerant']['X_c_r'].append(ASHPB.X_c_r)
    Data['refrigerant']['X_r_ext'].append(ASHPB.X_r_ext)
    Data['refrigerant']['X_r_HX'].append(ASHPB.X_r_exch)
    Data['refrigerant']['E_cmp'].append(ASHPB.E_cmp)   
    Data['refrigerant']['Q_r_ext'].append(ASHPB.Q_r_ext)
    Data['refrigerant']['Q_r_HX'].append(ASHPB.Q_r_exch)
    
    # Mixing valve exergy efficiency: X_w_serv / (X_w_exch + X_w_sup_mix)
    Data['heat exchanger']['X_eff'].append((ASHPB.X_w_exch / (ASHPB.X_r_exch + ASHPB.X_w_sup_exch)) * 100)
    Data['heat exchanger']['X_c_HX'].append(ASHPB.X_c_exch)
    Data['heat exchanger']['X_r_HX'].append(ASHPB.X_r_exch)
    Data['heat exchanger']['X_w_HX'].append(ASHPB.X_w_exch)
    Data['heat exchanger']['X_w_sup_HX'].append(ASHPB.X_w_sup_exch)
    Data['heat exchanger']['Q_w_HX'].append(ASHPB.Q_w_exch)
    Data['heat exchanger']['Q_r_HX'].append(ASHPB.Q_r_exch)
    Data['heat exchanger']['Q_w_sup_HX'].append(ASHPB.Q_w_sup_exch)

    # Mixing valve
    Data['mixing valve']['X_eff'].append((ASHPB.X_w_serv / (ASHPB.X_w_exch + ASHPB.X_w_sup_mix)) * 100)
    Data['mixing valve']['X_c_mix'].append(ASHPB.X_c_mix)
    Data['mixing valve']['X_w_serv'].append(ASHPB.X_w_serv)
    Data['mixing valve']['X_w_HX'].append(ASHPB.X_w_exch)
    Data['mixing valve']['X_w_sup_mix'].append(ASHPB.X_w_sup_mix)
    Data['mixing valve']['Q_w_serv'].append(ASHPB.Q_w_serv)
    Data['mixing valve']['Q_w_HX'].append(ASHPB.Q_w_exch)
    Data['mixing valve']['Q_w_sup_mix'].append(ASHPB.Q_w_sup_mix)

    # Whole
    Data['whole']['Ex_eff'].append(ASHPB.X_eff*100)
    Data['whole']['En_eff'].append((ASHPB.Q_w_serv/(ASHPB.E_fan + ASHPB.E_cmp))*100)
    Data['whole']['COP'].append(ASHPB.COP)
    Data['whole']['PLR'].append(ASHPB.Q_r_exch / ASHPB.Q_r_max * 100)

for component in Data:
    for key in Data[component]:
        Data[component][key] = np.array(Data[component][key])

print(np.max(Data['refrigerant']['X_eff']))
print(np.max((T0_range)*0.2 + 10), np.min((T0_range)*0.2 + 10))

80.59054717502441
12.0 6.0


# Figure

## Figure setting

In [4]:
def fs_dict_update(fs):
    plt.rcParams['font.size'] = fs

    fs_dict = {
        'label': dm.fs(0),
        'tick': dm.fs(-1),
        'ctick': dm.fs(-2),
        'legend': dm.fs(-1.5),
        'annotation': dm.fs(-1),
                }
    return fs_dict

pad = {
    'label': 6,
    'tick': 4,
}

layout ={
    'bbox': (0.1, 0.1, 0.8, 0.8),
    'margins': (0, 0, 0, 0),
}

LW = np.arange(0.25, 3.0, 0.25)

## Figure function

In [5]:
def draw_plots(
    datas,
    plot_labels =None,
    ylabel = None,
    ymin=None,
    ymax=None,
    yint=None,
    line_colors = None,
    line_styles = None,
    annotations=None,
    fig_name=None,
    save=None,
    fs_dict = fs_dict_update(10)
):
    """
    Generates and displays a figure with two side-by-side heatmaps, each with its own colorbar.
    
    Most parameters have default values and can be omitted unless customization is needed.
    """
    # Set default values for optional parameters if they are not provided
    DL = len(datas)
    if plot_labels is None: plot_labels = [f"Data {i+1}" for i in range(DL)]
    if ylabel is None: ylabel = 'Y label'
    if ymax is None: ymax = np.max([np.max(d) for d in datas])
    if ymin is None: ymin = np.min([np.min(d) for d in datas])
    if yint is None: yint = (ymax-ymin)/5

    if line_colors is None: line_colors = ['dm.gray' + str(i) for i in range(DL)]
    if line_styles is None: line_styles = ['-' for _ in range(DL)]
    if annotations is None: annotations = ['(a)' for _ in range(DL)]
    if fig_name is None: fig_name = 'default_figure'
    if save is None: save = False
    
    # Plot ====================================================================================================
    nrows = 1
    ncols = 1

    fig, ax = plt.subplots(
        nrows, ncols, 
        sharex=True, sharey=True, 
        figsize=(dm.DW, dm.SW),  # 가로 두배
        squeeze=True,
    )

    # x축을 T_w_serv, y축을 T0로 변경
    xmin = np.min(T0_range)
    xmax = T0_range[-1]
    xint = 10
    xmar = 0

    ymar = 0

    for k in range(DL):
        ax.plot(
            T0_range,
            datas[k],
            label = plot_labels[k],
            color = line_colors[k],
            linestyle = line_styles[k],
            linewidth = LW[3],
            zorder = 10 + k,  # Ensure lines are drawn in the order they are added
        )
    dm.simple_layout(fig, bbox=[0.08, 0.99, 0.1, 0.95], margins=[0.0, 0.1, 0.05, 0.05])
    
    ax.set_xlabel('Environmental temperature [°C]', fontsize=fs_dict['label'], labelpad=pad['label'])
    ax.set_ylabel(ylabel, fontsize=fs_dict['label'], labelpad=pad['label'])

    ax.set_xlim(xmin - xmar, xmax + xmar)
    ax.set_ylim(ymin - ymar, ymax + ymar)
    
    ax.annotate(annotations, xy=(.01, 1.01), xycoords='axes fraction',
        horizontalalignment='left', verticalalignment='bottom', fontsize=fs_dict['annotation']) 
    
    ax.set_xticks(np.arange(xmin, xmax*1.001, xint))
    ax.set_yticks(np.arange(ymin, ymax*1.001, yint))
    
    ax.tick_params(direction='in', labelsize=fs_dict['tick'], which='major', length=2.5, width=0.5  , pad=pad['tick'])
    ax.tick_params(direction='in', labelsize=fs_dict['tick'], which='minor', length=1.25, width=0.25, pad=pad['tick'])
    
    ax.xaxis.set_minor_locator(ticker.AutoMinorLocator(5))
    ax.yaxis.set_minor_locator(ticker.AutoMinorLocator(2))
    
    ax.grid(False)

    handles, labels = ax.get_legend_handles_labels()
    ax.legend(
        handles,
        labels,
        loc='upper left',
        fontsize=fs_dict['legend'],
        bbox_to_anchor=(0, 1),
        ncols=2,
        handlelength= 2,  # shorter handle
        handletextpad=0.6, # space between handle and text
        columnspacing=1.5, # space between columns
        borderaxespad=0.8  # padding around legend
    )

    save_path = 'figure/'
    
    if save:
        plt.savefig(save_path + fig_name + '.png', dpi=600)
        plt.savefig(save_path + fig_name + '.svg', transparent=True)
        
    dm.save_and_show(fig)
    plt.close()

## PLR

In [16]:
draw_plots(
    datas=[Data['whole']['PLR']],
    plot_labels=['PLR'],
    ylabel='PLR [-]',
    ymin=0,
    ymax=100,
    yint=20,
    line_colors=['dm.blue6'],
    line_styles=['-'],
    fig_name='ASHPB_COP&PLR_plots',
    annotations=' ',
    save=False
)

## COP

In [17]:
draw_plots(
    datas=[Data['whole']['COP']],
    plot_labels=['COP'],
    ylabel='COP [-]',
    ymin=0,
    ymax=10,
    yint=2,
    line_colors=['dm.green6'],
    line_styles=['-'],
    annotations=' ',
    fig_name='ASHPB_COP&PLR_plots',
    save=False
)

## Temperature

In [ ]:
T_w_serv
draw_plots(datas=[
           Data['temp']['T0'],
           Data['temp']['T_w_sup'],
           Data['temp']['T_w_serv'],
           Data['temp']['T_w_HX'],
           Data['temp']['T_r_HX'],],
    plot_labels=['T0', 'T_w_sup','T_w_serv', 'T_w_HX', 'T_r_HX'],
    ylabel='Temperature [°C]',
    ymin=-20,
    ymax=80,
    yint=20,
    line_colors=['dm.blue6','dm.lime6','dm.red6','dm.green6','dm.yellow6'],
    line_styles=['-.','-','-.',':', '--'],
    annotations=' ',
    fig_name='ASHPB_COP&PLR_plots',
    save=False
)

## Exergy In Out

### External unit

In [111]:
T_w_serv
draw_plots(datas=[
           Data['external unit']['X_a_ext_out'],
           Data['external unit']['X_a_ext_in'],
        #    Data['external unit']['E_fan'],
           ],
    plot_labels=['X_a_ext_out', 'X_a_ext_in', 'E_fan'],
    ylabel='Exergy (external unit) [W]',
    ymin=0,
    ymax=120,
    yint=20,
    line_colors=['dm.blue6','dm.lime6','dm.red6','dm.green6','dm.yellow6'],
    line_styles=['-.','-','-.',':', '--'],
    annotations=' ',
    fig_name='ASHPB_COP&PLR_plots',
    save=False
)

### Refrigerant

In [ ]:
T_w_serv
draw_plots(datas=[
           Data['refrigerant']['X_r_ext'],
           Data['refrigerant']['X_r_HX'],
           Data['refrigerant']['E_cmp'],
           ],
    plot_labels=['X_r_ext', 'X_r_HX', 'X_cmp'],
    ylabel='Exergy (refrigerant) [W]',
    ymin=0,
    ymax=12000,
    yint=2000,
    line_colors=['dm.blue6','dm.lime6','dm.red6','dm.green6','dm.yellow6'],
    line_styles=['-.','-','-.',':', '--'],
    annotations=' ',
    fig_name='ASHPB_COP&PLR_plots',
    save=False
)

In [ ]:
T_w_serv
draw_plots(datas=[
           Data['refrigerant']['E_cmp']-Data['refrigerant']['X_r_ext']-Data['refrigerant']['X_r_HX']
           ],
    plot_labels=['X_cmp - X_r_ext - X_r_HX'],
    ylabel='Exergy (refrigerant) [W]',
    ymin=0,
    ymax=12000,
    yint=2000,
    line_colors=['dm.blue6','dm.lime6','dm.red6','dm.green6','dm.yellow6'],
    line_styles=['-.','-','-.',':', '--'],
    annotations=' ',
    fig_name='ASHPB_COP&PLR_plots',
    save=False
)

### Heat exchanger

In [ ]:
draw_plots(datas=[
           Data['heat exchanger']['X_w_HX'],
           Data['heat exchanger']['X_w_sup_HX'],
           ],
    plot_labels=['X_w_HX', 'X_w_sup_HX'],
    ylabel='Exergy (heat exchanger) [W]',
    ymin=0,
    ymax=4000,
    yint=1000,
    line_colors=['dm.blue6','dm.lime6','dm.red6','dm.green6','dm.yellow6'],
    line_styles=['-.','-','-.',':', '--'],
    annotations=' ',
    fig_name='ASHPB_COP&PLR_plots',
    save=False
)

### Mixing valve

In [ ]:
draw_plots(datas=[
           Data['mixing valve']['X_w_HX'],
           Data['mixing valve']['X_w_sup_mix'],
           Data['mixing valve']['X_w_serv'],
           ],
    plot_labels=['X_w_HX', 'X_w_sup_mix', 'X_w_serv'],
    ylabel='Exergy (mixing valve) [W]',
    ymin=0,
    ymax=4000,
    yint=1000,
    line_colors=['dm.blue6','dm.lime6','dm.red6','dm.green6','dm.yellow6'],
    line_styles=['-.','-','-.',':', '--'],
    annotations=' ',
    fig_name='ASHPB_COP&PLR_plots',
    save=False
)

## System energy, exergy efficiency

In [87]:
fs_dict = fs_dict_update(10)
fig, ax1 = plt.subplots(figsize=(dm.DW, dm.cm2in(8)))

color1 = 'dm.gray6'
color2 = 'dm.red6'

# 첫 번째 데이터: Energy efficiency
ax1.plot(T0_range, Data['whole']['En_eff'], color=color1, label='Energy efficiency', linewidth=LW[3])

# 두 번째 데이터: Exergy efficiency
ax2 = ax1.twinx()
ax2.plot(T0_range, Data['whole']['Ex_eff'], color=color2, label='Exergy efficiency', linewidth=LW[3])

# fill_between: 두 축의 값을 같은 축에서 그려야 하므로, 보조축(ax2) 데이터를 ax1의 축에 맞게 변환 필요
# 여기서는 단순히 두 곡선 사이를 채우는 시각적 효과를 위해, 두 곡선을 같은 축(ax1)에 임시로 그려서 fill_between 사용
# ax1.fill_between(T0_range, Data['whole']['En_eff'], 0, color=color1, alpha=0.3)
# ax2.fill_between(T0_range, Data['whole']['Ex_eff'], 0, color=color2, alpha=0.3)

# 최고점 위치에 수직선 추가
max_en_idx = np.argmax(Data['whole']['En_eff'])
max_ex_idx = np.argmax(Data['whole']['Ex_eff'])
max_en_x = T0_range[max_en_idx]
max_ex_x = T0_range[max_ex_idx]
max_en_y = Data['whole']['En_eff'][max_en_idx]
max_ex_y = Data['whole']['Ex_eff'][max_ex_idx]

print('max_en_eff_T0:', round(max_en_x,1))
print('max_ex_eff_T0:', round(max_ex_x,1))

ymin1, ymax1 = 250, 500
ymin2, ymax2 = 15, 40

ss = 30  # scatter size
ax1.scatter(max_en_x, max_en_y, s=ss, color=color1, edgecolor='white', linewidth=LW[3], zorder=5)
ax2.scatter(max_ex_x, max_ex_y, s=ss, color=color2, edgecolor='white', linewidth=LW[3], zorder=5)

ax1.set_xlabel('Environmental temperature [°C]', fontsize=fs_dict['label'], labelpad=pad['label'])
ax1.set_ylabel('System energy efficiency [%]', color=color1, fontsize=fs_dict['label'], labelpad=pad['label'])

ax1.tick_params(axis='x', colors='k', labelsize=fs_dict['tick'], pad=pad['tick'])
ax1.tick_params(axis='y', colors=color1, labelsize=fs_dict['tick'], pad=pad['tick'])

ax1.set_xlim(np.min(T0_range), T0_range[-1])
ax1.set_ylim(ymin1, ymax1)

ax1.set_xticks(np.arange(np.min(T0_range), T0_range[-1]+1, 5))
ax1.set_yticks(np.arange(ymin1, ymax1+1, 50))

ax2.set_ylabel('System exergy efficiency [%]', color=color2, fontsize=fs_dict['label'], labelpad=pad['label'])
ax2.tick_params(axis='y', colors=color2, labelsize=fs_dict['tick'], pad=pad['tick'])
ax2.set_ylim(ymin2, ymax2)
ax2.set_yticks(np.arange(ymin2, ymax2+1, 5))

# --- 이제 축 한계가 정해졌으니, 수직선을 '점까지' 제한 ---
ymin1, ymax1 = ax1.get_ylim()
ymin2, ymax2 = ax2.get_ylim()

# 각 축에서 최대 y가 차지하는 비율(0~1)
frac_en = (max_en_y - ymin1) / (ymax1 - ymin1)
frac_ex = (max_ex_y - ymin2) / (ymax2 - ymin2)

# 수직선: 점까지 그리되, zorder는 scatter보다 낮게
ax1.axvline(x=max_en_x, ymin=0.0, ymax=frac_en, color=color1, linestyle='--', linewidth=LW[2], zorder=2)
ax2.axvline(x=max_ex_x, ymin=0.0, ymax=frac_ex, color=color2, linestyle='--', linewidth=LW[2], zorder=2)

for k in ['left', 'top','bottom']:
    ax2.spines[k].set_visible(False)
ax2.spines['right'].set_visible(True)

# spine color
ax1.spines['left'].set_color(color1)
ax2.spines['right'].set_color(color2)

# 범례 추가
lines_1, labels_1 = ax1.get_legend_handles_labels()
lines_2, labels_2 = ax2.get_legend_handles_labels()

ax1.legend(
    lines_1 + lines_2,
    labels_1 + labels_2,
    loc='upper left',
    fontsize=fs_dict['legend'],
    bbox_to_anchor=(0, 1.05),
    ncols=1,
    handlelength= 2,  # shorter handle
    handletextpad=0.6, # space between handle and text
    columnspacing=1.5, # space between columns
    borderaxespad=0.8  # padding around legend
)

print('에너지 최대 효율 지점에서의 에너지 효율:', round(Data['whole']['En_eff'][max_en_idx], 1), '%')
print('에너지 최대 효율 지점에서의 엑서지 효율:', round(Data['whole']['Ex_eff'][max_en_idx], 1), '%')
print('')
print('엑서지 최대 효율 지점에서의 에너지 효율:', round(Data['whole']['En_eff'][max_ex_idx], 1), '%')
print('엑서지 최대 효율 지점에서의 엑서지 효율:', round(Data['whole']['Ex_eff'][max_ex_idx], 1), '%')


dm.simple_layout(fig, bbox=[0.01, 0.99, 0.01, 1.0], margins=[0.0, 0.1, 0.05, 0.05])
plt.savefig('figure/ASHPB_en_ex_eff_comparison.svg', transparent=True)
plt.savefig('figure/ASHPB_en_ex_eff_comparison.png',dpi=600)
dm.save_and_show(fig)
plt.close()

max_en_eff_T0: 2.7
max_ex_eff_T0: -9.4
에너지 최대 효율 지점에서의 에너지 효율: 480.7 %
에너지 최대 효율 지점에서의 엑서지 효율: 29.8 %

엑서지 최대 효율 지점에서의 에너지 효율: 427.1 %
엑서지 최대 효율 지점에서의 엑서지 효율: 35.6 %


## Subsystem exergy efficiency

In [39]:
fs_dict = fs_dict_update(12)

# label
xlabel = 'Environmental temperature [°C]'
ylabel = 'Exergy efficiency [%]'

#
max_en_idx = np.argmax(Data['whole']['En_eff'])
max_ex_idx = np.argmax(Data['whole']['Ex_eff'])
max_en_x = T0_range[max_en_idx]
max_ex_x = T0_range[max_ex_idx]

# min max
xmin = -20; xmax = 10; xint = 10; xmar = 0
ymin = 0; ymax = 100; yint = 20; ymar = 0 

# 1) Figure & GridSpec 만들기
fig = plt.figure(figsize=(dm.DW, dm.cm2in(8)))
gs = fig.add_gridspec(nrows=1, ncols=1, left=0.1, right=0.95, top=0.9, bottom=0.15, hspace=0.2, wspace=0.25)

# 2) 서브플롯(axes) 생성
ax = fig.add_subplot(gs[0, 0])  # 좌상

# 3) (선택) 기본 제목/격자 등 틀만 잡아두기
ax.grid(True, linestyle = ':', linewidth = LW[1], color = 'dm.gray3')  # 보기 좋게 격자만 켜둠

# 4) plot
plot1 = ax.plot(T0_range, Data['external unit']['X_eff'], color='dm.lime6', label='external unit', linewidth=LW[3])
plot2 = ax.plot(T0_range, Data['refrigerant']['X_eff'], color='dm.blue6', label='refrigerant', linewidth=LW[3])
plot3 = ax.plot(T0_range, Data['heat exchanger']['X_eff'], color='dm.red6', label='heat exchanger', linewidth=LW[3])
plot4 = ax.plot(T0_range, Data['mixing valve']['X_eff'], color='dm.orange6', label='mixing valve', linewidth=LW[3])

ax.axvline(x=max_en_x, ymin=0.0, ymax=1.0, color='dm.gray6', linestyle='--', linewidth=LW[1], zorder=2)
ax.axvline(x=max_ex_x, ymin=0.0, ymax=1.0, color='dm.gray6', linestyle='--', linewidth=LW[1], zorder=2)

ss = 20 # scatter size
selw = LW[2] # scatter edge line width
ax.scatter(max_en_x, Data['external unit']['X_eff'][max_en_idx], s=ss, color='dm.lime6', edgecolor='white', linewidth=selw, zorder=5)
ax.scatter(max_en_x, Data['refrigerant']['X_eff'][max_en_idx], s=ss, color='dm.blue6', edgecolor='white', linewidth=selw, zorder=5)
ax.scatter(max_en_x, Data['heat exchanger']['X_eff'][max_en_idx], s=ss, color='dm.red6', edgecolor='white', linewidth=selw, zorder=5)
ax.scatter(max_en_x, Data['mixing valve']['X_eff'][max_en_idx], s=ss, color='dm.orange6', edgecolor='white', linewidth=selw, zorder=5)

ax.scatter(max_ex_x, Data['external unit']['X_eff'][max_ex_idx], s=ss, color='dm.lime6', edgecolor='white', linewidth=selw, zorder=5)
ax.scatter(max_ex_x, Data['refrigerant']['X_eff'][max_ex_idx], s=ss, color='dm.blue6', edgecolor='white', linewidth=selw, zorder=5)
ax.scatter(max_ex_x, Data['heat exchanger']['X_eff'][max_ex_idx], s=ss, color='dm.red6', edgecolor='white', linewidth=selw, zorder=5)
ax.scatter(max_ex_x, Data['mixing valve']['X_eff'][max_ex_idx], s=ss, color='dm.orange6', edgecolor='white', linewidth=selw, zorder=5)

# 5) plot detail
ax.set_xlabel('Environmental temperature [°C]', fontsize=fs_dict['label'], labelpad=pad['label'])
ax.set_ylabel(ylabel, fontsize=fs_dict['label'], labelpad=pad['label'])

ax.set_xlim(xmin - xmar, xmax + xmar)
ax.set_ylim(ymin - ymar, ymax + ymar)

# ax.annotate(annotations, xy=(.01, 1.01), xycoords='axes fraction',
#     horizontalalignment='left', verticalalignment='bottom', fontsize=fs_dict['annotation']) 

ax.set_xticks(np.arange(xmin, xmax*1.001, xint))
ax.set_yticks(np.arange(ymin, ymax*1.001, yint))

ax.tick_params(direction='in', labelsize=fs_dict['tick'], which='major', length=2.5, width=0.5  , pad=pad['tick'])
ax.tick_params(direction='in', labelsize=fs_dict['tick'], which='minor', length=1.25, width=0.25, pad=pad['tick'])

ax.xaxis.set_minor_locator(ticker.AutoMinorLocator(5))
ax.yaxis.set_minor_locator(ticker.AutoMinorLocator(2))

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, labels, loc='lower center', fontsize=fs_dict['legend'], bbox_to_anchor=(0.5, 1), ncols=4,
          handlelength = 1.5, columnspacing=2)

plt.savefig('figure/ASHPB_subsystem_ex_eff.svg', transparent=True)
plt.savefig('figure/ASHPB_subsystem_ex_eff.png',dpi=600)
dm.save_and_show(fig)
plt.close()

## Subsystem energy results

In [86]:
# --- Your original setup code ---
max_en_idx = np.argmax(Data['whole']['En_eff'])
max_ex_idx = np.argmax(Data['whole']['Ex_eff'])
max_en_x = T0_range[max_en_idx]
max_ex_x = T0_range[max_ex_idx]

fs_dict = fs_dict_update(8)
W2kW = 1/1000

# NEW: Create a dictionary to store output strings before printing
print_output = {
    "(a) External unit": [],
    "(b) Refrigerant": [],
    "(c) Heat exchanger": [],
    "(d) Mixing valve": []
}

# 0) min max
ymin = [-16, 0, 0, 0]
ymax = [16, 20, 30, 40]
yint = [8, 5, 10, 10]

# 1) Figure & GridSpec 만들기
fig = plt.figure(figsize=(dm.DW, dm.cm2in(10)))
gs = fig.add_gridspec(nrows=2, ncols=2, left=0.07, right=0.97, top=0.95, bottom=0.1, hspace=0.4, wspace=0.2)

# 2) 서브플롯(axes) 생성
ax1 = fig.add_subplot(gs[0, 0])  # 좌상
ax2 = fig.add_subplot(gs[0, 1])  # 우상
ax3 = fig.add_subplot(gs[1, 0])  # 좌하
ax4 = fig.add_subplot(gs[1, 1])  # 우하

for i, ax in enumerate((ax1, ax2, ax3, ax4)):
    ax.axvline(x=max_en_x, ymin=0.0, ymax=1.0, color='dm.gray9', linestyle='--', linewidth=LW[1], zorder=2)
    ax.axvline(x=max_ex_x, ymin=0.0, ymax=1.0, color='dm.red6', linestyle='--', linewidth=LW[1], zorder=2)

ss = 10 # scatter size
selw = LW[1] # scatter edge line width

# --- Loop for ax1: Store data for later printing ---
key = "(a) External unit"
# CORRECTED: Matched list lengths (4 items each) for zip to work correctly.
for data, col, label, ls in zip([Data['external unit']['Q_a_ext_in'], Data['external unit']['E_fan'], Data['external unit']['Q_r_ext'], Data['external unit']['Q_a_ext_out']],
                                 ['dm.green4', 'dm.green4', 'dm.orange4', 'dm.orange4'],
                                 ['$Q_{a,ext,in}$', '$E_{fan}$', '$Q_{r,ext}$', '$Q_{a,ext,out}$'],
                                 ['-', ':', '-', ':']):
    ax1.plot(T0_range, data*W2kW, color=col, label=label, linewidth=LW[2], linestyle=ls)
    en_y_val = data[max_en_idx]*W2kW
    ex_y_val = data[max_ex_idx]*W2kW
    ax1.scatter(max_en_x, en_y_val, s=ss, color=col, edgecolor='white', linewidth=selw, zorder=5)
    ax1.scatter(max_ex_x, ex_y_val, s=ss, color=col, edgecolor='white', linewidth=selw, zorder=5)
    # NEW: Format string and add to dictionary
    output_string = f"{label.replace('$', ''):<15} -> Max En Eff Point: ({max_en_x:>5.2f}, {en_y_val:.4f}), Max Ex Eff Point: ({max_ex_x:>5.2f}, {ex_y_val:.4f})"
    print_output[key].append(output_string)

# --- Loop for ax2: Store data for later printing ---
key = "(b) Refrigerant"
# CORRECTED: Matched list lengths (3 items each).
for data, col, label, ls in zip([Data['refrigerant']['E_cmp'], Data['refrigerant']['Q_r_ext'], Data['refrigerant']['Q_r_HX']],
                                 ['dm.green4', 'dm.green4', 'dm.orange4'],
                                 ['$E_{cmp}$', '$Q_{r,ext}$', '$Q_{r,HX}$'],
                                 ['-', ':', '-']):
    ax2.plot(T0_range, data*W2kW, color=col, label=label, linewidth=LW[2], linestyle=ls)
    en_y_val = data[max_en_idx]*W2kW
    ex_y_val = data[max_ex_idx]*W2kW
    ax2.scatter(max_en_x, en_y_val, s=ss, color=col, edgecolor='white', linewidth=selw, zorder=5)
    ax2.scatter(max_ex_x, ex_y_val, s=ss, color=col, edgecolor='white', linewidth=selw, zorder=5)
    # NEW: Format string and add to dictionary
    output_string = f"{label.replace('$', ''):<15} -> Max En Eff Point: ({max_en_x:>5.2f}, {en_y_val:.4f}), Max Ex Eff Point: ({max_ex_x:>5.2f}, {ex_y_val:.4f})"
    print_output[key].append(output_string)

# --- Loop for ax3: Store data for later printing ---
key = "(c) Heat exchanger"
# CORRECTED: Matched list lengths (3 items each).
for data, col, label, ls in zip([Data['heat exchanger']['Q_r_HX'], Data['heat exchanger']['Q_w_sup_HX'], Data['heat exchanger']['Q_w_HX']],
                                  ['dm.green4', 'dm.green4', 'dm.orange4'],
                                  ['$Q_{r,HX}$', '$Q_{w,sup,HX}$', '$Q_{w,HX}$'],
                                  ['-', ':', '-']):
    ax3.plot(T0_range, data*W2kW, color=col, label=label, linewidth=LW[2], linestyle=ls)
    en_y_val = data[max_en_idx]*W2kW
    ex_y_val = data[max_ex_idx]*W2kW
    ax3.scatter(max_en_x, en_y_val, s=ss, color=col, edgecolor='white', linewidth=selw, zorder=5)
    ax3.scatter(max_ex_x, ex_y_val, s=ss, color=col, edgecolor='white', linewidth=selw, zorder=5)
    # NEW: Format string and add to dictionary
    output_string = f"{label.replace('$', ''):<15} -> Max En Eff Point: ({max_en_x:>5.2f}, {en_y_val:.4f}), Max Ex Eff Point: ({max_ex_x:>5.2f}, {ex_y_val:.4f})"
    print_output[key].append(output_string)

# --- Loop for ax4: Store data for later printing ---
key = "(d) Mixing valve"
# CORRECTED: Matched list lengths (3 items each).
for data, col, label, ls in zip([Data['mixing valve']['Q_w_HX'], Data['mixing valve']['Q_w_sup_mix'], Data['mixing valve']['Q_w_serv']],
                                  ['dm.green4', 'dm.green4', 'dm.orange4'],
                                  ['$Q_{w,HX}$', '$Q_{w,sup,mix}$', '$Q_{w,serv}$'],
                                  ['-', ':', '-']):
    ax4.plot(T0_range, data*W2kW, color=col, label=label, linewidth=LW[2], linestyle=ls)
    en_y_val = data[max_en_idx]*W2kW
    ex_y_val = data[max_ex_idx]*W2kW
    ax4.scatter(max_en_x, en_y_val, s=ss, color=col, edgecolor='white', linewidth=selw, zorder=5)
    ax4.scatter(max_ex_x, ex_y_val, s=ss, color=col, edgecolor='white', linewidth=selw, zorder=5)
    # NEW: Format string and add to dictionary
    output_string = f"{label.replace('$', ''):<15} -> Max En Eff Point: ({max_en_x:>5.2f}, {en_y_val:.4f}), Max Ex Eff Point: ({max_ex_x:>5.2f}, {ex_y_val:.4f})"
    print_output[key].append(output_string)

# --- Your original figure formatting and saving code ---
an_head = ['a', 'b', 'c', 'd']
an_main = ['External unit', 'Refrigerant', 'Ref-to-water heat exchanger', 'Mixing valve']

for i, ax in enumerate((ax1, ax2, ax3, ax4)):
    ax.annotate(an_head[i], xy=(0, 1.04), xycoords='axes fraction',
                fontsize=fs_dict['annotation'], ha='left', va='bottom', fontweight = 600)
    ax.annotate(an_main[i], xy=(0.04, 1.04), xycoords='axes fraction',
                fontsize=fs_dict['annotation'], ha='left', va='bottom')
    handles, labels = ax.get_legend_handles_labels()
    ax.legend(handles, labels, loc='upper right', fontsize=fs_dict['legend'], bbox_to_anchor=(1, 1), ncols=6,
        handlelength = 1.2, columnspacing=1, )

    ymar = (ymax[i] - ymin[i]) / 30
    ax.set_ylim(ymin[i] - ymar, ymax[i])
    ax.set_yticks(np.arange(ymin[i], ymax[i]*1.001, yint[i]))
    ax.tick_params(axis='both', which='major', labelsize=fs_dict['tick'])

fig.supxlabel("Environmental temperature [°C]", fontsize=fs_dict['label'], y = 0.005, fontweight = 400)
# Note: You might want to change this label to "Energy rate [kW]" if it's no longer Exergy
fig.supylabel("Exergy rate [kW]", fontsize=fs_dict['label'], x = 0.005, fontweight = 400)

for ax in (ax1, ax2, ax3, ax4):
    ax.set_axisbelow(True)
    ax.grid(True, linestyle=':', linewidth=LW[1], color='dm.gray3', zorder = 0)

# plt.show()
plt.savefig('figure/ASHPB_subsystem_en_in_out.svg', transparent=True)
plt.savefig('figure/ASHPB_subsystem_en_in_out.png',dpi=600)
dm.save_and_show(fig)
plt.close()

# --- NEW: Print all stored information at the very end ---
print("--- Scatter Point Data Summary ---")
for subplot_title, lines in print_output.items():
    print(f"\n--- Subplot {subplot_title} ---")
    for line in lines:
        print(line)

--- Scatter Point Data Summary ---

--- Subplot (a) External unit ---
Q_{a,ext,in}    -> Max En Eff Point: ( 2.73, 0.0000), Max Ex Eff Point: (-9.39, 0.0000)
E_{fan}         -> Max En Eff Point: ( 2.73, 0.6149), Max Ex Eff Point: (-9.39, 0.5756)
Q_{r,ext}       -> Max En Eff Point: ( 2.73, 11.9695), Max Ex Eff Point: (-9.39, 11.2048)
Q_{a,ext,out}   -> Max En Eff Point: ( 2.73, -11.3547), Max Ex Eff Point: (-9.39, -10.6292)

--- Subplot (b) Refrigerant ---
E_{cmp}         -> Max En Eff Point: ( 2.73, 3.4425), Max Ex Eff Point: (-9.39, 5.4758)
Q_{r,ext}       -> Max En Eff Point: ( 2.73, 11.9695), Max Ex Eff Point: (-9.39, 11.2048)
Q_{r,HX}        -> Max En Eff Point: ( 2.73, 15.4121), Max Ex Eff Point: (-9.39, 16.6806)

--- Subplot (c) Heat exchanger ---
Q_{r,HX}        -> Max En Eff Point: ( 2.73, 15.4121), Max Ex Eff Point: (-9.39, 16.6806)
Q_{w,sup,HX}    -> Max En Eff Point: ( 2.73, 3.4972), Max Ex Eff Point: (-9.39, 7.9222)
Q_{w,HX}        -> Max En Eff Point: ( 2.73, 18.9093), Ma

## Subsystem exergy results

In [ ]:

# --- Your original setup code ---
max_en_idx = np.argmax(Data['whole']['En_eff'])
max_ex_idx = np.argmax(Data['whole']['Ex_eff'])
max_en_x = T0_range[max_en_idx]
max_ex_x = T0_range[max_ex_idx]


fs_dict = fs_dict_update(8)
W2kW = 1/1000

# 0) min max
ymin = [0, 0, 0, 0]
ymax = [1.2, 12, 4, 4]
yint = [0.3, 3, 1, 1]

# NEW: Create a dictionary to store output strings before printing
print_output = {
    "(a) External unit": [],
    "(b) Refrigerant": [],
    "(c) Heat exchanger": [],
    "(d) Mixing valve": []
}

# 1) Figure & GridSpec 만들기
fig = plt.figure(figsize=(dm.DW, dm.cm2in(10)))
gs = fig.add_gridspec(nrows=2, ncols=2, left=0.07, right=0.97, top=0.95, bottom=0.1, hspace=0.4, wspace=0.2)

# 2) 서브플롯(axes) 생성
ax1 = fig.add_subplot(gs[0, 0])
ax2 = fig.add_subplot(gs[0, 1])
ax3 = fig.add_subplot(gs[1, 0])
ax4 = fig.add_subplot(gs[1, 1])

for i, ax in enumerate((ax1, ax2, ax3, ax4)):
    ax.axvline(x=max_en_x, ymin=0.0, ymax=1.0, color='dm.gray6', linestyle='--', linewidth=LW[1], zorder=2)
    ax.axvline(x=max_ex_x, ymin=0.0, ymax=1.0, color='dm.red6', linestyle='--', linewidth=LW[1], zorder=2)


ss = 10
selw = LW[1]

# --- Loop for ax1: Store data for later printing ---
key = "(a) External unit"
for data, col, label, ls in zip([Data['external unit']['X_a_ext_in'], Data['external unit']['X_r_ext'], Data['external unit']['E_fan'], Data['external unit']['X_a_ext_out'], Data['external unit']['X_c_ext']],
                                 ['dm.green4', 'dm.green4', 'dm.green4', 'dm.orange4', 'dm.violet4'],
                                 ['$X_{a,ext,in}$', '$X_{r,ext}$', '$X_{fan}$', '$X_{a,ext,out}$', '$X_{c,ext}$'],
                                 ['-', ':', '-.', '-', '-']):
    ax1.plot(T0_range, data*W2kW, color=col, label=label, linewidth=LW[2], linestyle=ls)
    en_y_val = data[max_en_idx]*W2kW
    ex_y_val = data[max_ex_idx]*W2kW
    ax1.scatter(max_en_x, en_y_val, s=ss, color=col, edgecolor='white', linewidth=selw, zorder=5)
    ax1.scatter(max_ex_x, ex_y_val, s=ss, color=col, edgecolor='white', linewidth=selw, zorder=5)
    # NEW: Format string and add to dictionary instead of printing
    output_string = f"{label.replace('$', ''):<15} -> Max En Eff Point: ({max_en_x:>5.2f}, {en_y_val:.4f}), Max Ex Eff Point: ({max_ex_x:>5.2f}, {ex_y_val:.4f})"
    print_output[key].append(output_string)

# --- Loop for ax2: Store data for later printing ---
key = "(b) Refrigerant"
for data, col, label, ls in zip([Data['refrigerant']['E_cmp'], Data['refrigerant']['X_r_ext'], Data['refrigerant']['X_r_HX'], Data['refrigerant']['X_c_r']],
                                 ['dm.green4', 'dm.orange4', 'dm.orange4', 'dm.violet4'],
                                 ['$X_{cmp}$', '$X_{r,ext}$', '$X_{r,HX}$', '$X_{c,r}$'],
                                 ['-', '-', ':', '-']):
    ax2.plot(T0_range, data*W2kW, color=col, label=label, linewidth=LW[2], linestyle=ls)
    en_y_val = data[max_en_idx]*W2kW
    ex_y_val = data[max_ex_idx]*W2kW
    ax2.scatter(max_en_x, en_y_val, s=ss, color=col, edgecolor='white', linewidth=selw, zorder=5)
    ax2.scatter(max_ex_x, ex_y_val, s=ss, color=col, edgecolor='white', linewidth=selw, zorder=5)
    # NEW: Format string and add to dictionary
    output_string = f"{label.replace('$', ''):<15} -> Max En Eff Point: ({max_en_x:>5.2f}, {en_y_val:.4f}), Max Ex Eff Point: ({max_ex_x:>5.2f}, {ex_y_val:.4f})"
    print_output[key].append(output_string)

# --- Loop for ax3: Store data for later printing ---
key = "(c) Heat exchanger"
for data, col, label, ls in zip([Data['heat exchanger']['X_w_sup_HX'], Data['heat exchanger']['X_r_HX'], Data['heat exchanger']['X_w_HX'], Data['heat exchanger']['X_c_HX']],
                                  ['dm.green4', 'dm.green4', 'dm.orange4', 'dm.violet4'],
                                  ['$X_{w,sup,HX}$', '$X_{r,HX}$', '$X_{w,HX}$', '$X_{c,HX}$'],
                                  ['-', ':', '-', '-']):
    ax3.plot(T0_range, data*W2kW, color=col, label=label, linewidth=LW[2], linestyle=ls)
    en_y_val = data[max_en_idx]*W2kW
    ex_y_val = data[max_ex_idx]*W2kW
    ax3.scatter(max_en_x, en_y_val, s=ss, color=col, edgecolor='white', linewidth=selw, zorder=5)
    ax3.scatter(max_ex_x, ex_y_val, s=ss, color=col, edgecolor='white', linewidth=selw, zorder=5)
    # NEW: Format string and add to dictionary
    output_string = f"{label.replace('$', ''):<15} -> Max En Eff Point: ({max_en_x:>5.2f}, {en_y_val:.4f}), Max Ex Eff Point: ({max_ex_x:>5.2f}, {ex_y_val:.4f})"
    print_output[key].append(output_string)

# --- Loop for ax4: Store data for later printing ---
key = "(d) Mixing valve"
for data, col, label, ls in zip([Data['mixing valve']['X_w_HX'], Data['mixing valve']['X_w_sup_mix'], Data['mixing valve']['X_w_serv'], Data['mixing valve']['X_c_mix']],
                                  ['dm.green4', 'dm.green4', 'dm.orange4', 'dm.violet4'],
                                  ['$X_{w,HX}$', '$X_{w,sup,mix}$', '$X_{w,serv}$', '$X_{c,mix}$'],
                                  ['-', ':', '-', '-']):
    ax4.plot(T0_range, data*W2kW, color=col, label=label, linewidth=LW[2], linestyle=ls)
    en_y_val = data[max_en_idx]*W2kW
    ex_y_val = data[max_ex_idx]*W2kW
    ax4.scatter(max_en_x, en_y_val, s=ss, color=col, edgecolor='white', linewidth=selw, zorder=5)
    ax4.scatter(max_ex_x, ex_y_val, s=ss, color=col, edgecolor='white', linewidth=selw, zorder=5)
    # NEW: Format string and add to dictionary
    output_string = f"{label.replace('$', ''):<15} -> Max En Eff Point: ({max_en_x:>5.2f}, {en_y_val:.4f}), Max Ex Eff Point: ({max_ex_x:>5.2f}, {ex_y_val:.4f})"
    print_output[key].append(output_string)

# --- Your original figure formatting and saving code ---
an_head = ['a', 'b', 'c', 'd']
an_main =['External unit', 'Refrigerant', 'Ref-to-water heat exchanger', 'Mixing valve']

for i, ax in enumerate((ax1, ax2, ax3, ax4)):
    ax.annotate(an_head[i], xy=(0, 1.04), xycoords='axes fraction',
                fontsize=fs_dict['annotation'], ha='left', va='bottom', fontweight = 600)
    ax.annotate(an_main[i], xy=(0.04, 1.04), xycoords='axes fraction',
                fontsize=fs_dict['annotation'], ha='left', va='bottom')
    handles, labels = ax.get_legend_handles_labels()
    ax.legend(handles, labels, loc='upper right', fontsize=fs_dict['legend'], bbox_to_anchor=(1, 1), ncols=6,
        handlelength = 1.2, columnspacing=1)

    ymar = (ymax[i] - ymin[i]) / 30
    ax.set_ylim(ymin[i] - ymar, ymax[i])
    ax.set_yticks(np.arange(ymin[i], ymax[i]*1.001, yint[i]))
    ax.tick_params(axis='both', which='major', labelsize=fs_dict['tick'])

fig.supxlabel("Environmental temperature [°C]", fontsize=fs_dict['label'], y = 0.005, fontweight = 500)
fig.supylabel("Exergy rate [kW]", fontsize=fs_dict['label'], x = 0.005, fontweight = 500)

for ax in (ax1, ax2, ax3, ax4):
    ax.set_axisbelow(True)
    ax.grid(True, linestyle=':', linewidth=LW[1], color='dm.gray3', zorder = 0)

# plt.show()
plt.savefig('figure/ASHPB_subsystem_ex_in_out.svg', transparent=True)
plt.savefig('figure/ASHPB_subsystem_ex_in_out.png',dpi=600)
dm.save_and_show(fig)
plt.close()

# --- NEW: Print all stored information at the very end ---
print("--- Scatter Point Data Summary ---")
for subplot_title, lines in print_output.items():
    print(f"\n--- Subplot {subplot_title} ---")
    for line in lines:
        print(line)

--- Scatter Point Data Summary ---

--- Subplot (a) External unit ---
X_{a,ext,in}    -> Max En Eff Point: ( 2.73, 0.0000), Max Ex Eff Point: (-9.39, 0.0000)
X_{r,ext}       -> Max En Eff Point: ( 2.73, 0.4502), Max Ex Eff Point: (-9.39, 0.4416)
X_{fan}         -> Max En Eff Point: ( 2.73, 0.6149), Max Ex Eff Point: (-9.39, 0.5756)
X_{a,ext,out}   -> Max En Eff Point: ( 2.73, 0.1042), Max Ex Eff Point: (-9.39, 0.1020)
X_{c,ext}       -> Max En Eff Point: ( 2.73, 0.9609), Max Ex Eff Point: (-9.39, 0.9151)

--- Subplot (b) Refrigerant ---
X_{cmp}         -> Max En Eff Point: ( 2.73, 3.4425), Max Ex Eff Point: (-9.39, 5.4758)
X_{r,ext}       -> Max En Eff Point: ( 2.73, 0.4502), Max Ex Eff Point: (-9.39, 0.4416)
X_{r,HX}        -> Max En Eff Point: ( 2.73, 2.2546), Max Ex Eff Point: (-9.39, 3.0658)
X_{c,r}         -> Max En Eff Point: ( 2.73, 0.7378), Max Ex Eff Point: (-9.39, 1.9684)

--- Subplot (c) Heat exchanger ---
X_{w,sup,HX}    -> Max En Eff Point: ( 2.73, 0.0486), Max Ex Eff Poin

In [ ]:
ymin = np.array([0, 0, 0, 0])
ymax = np.array([1.2, 12, 4, 4])
yint = np.array([0.3, 3, 1, 1])
ymar = (ymax-ymin)/20
ymar[1]

np.float64(0.6)

## Water fall chart

In [ ]:
T0 = T0_range[np.argmax(Data['whole']['En_eff'])]  # T0 at max exergy efficiency
ASHPB = enex.HeatPumpBoiler_without_tank()
ASHPB.dV_w_serv = 7.5
ASHPB.Q_r_max = 20000

ASHPB.T0          = T0
ASHPB.T_w_serv    = T_w_serv
ASHPB.T_w_sup     = 0.2 * T0 + 10
ASHPB.T_a_ext_out = T0 - 5
ASHPB.T_r_ext     = T0 - 10

ASHPB.T_r_exch    = T_w_serv + 10
ASHPB.T_w_exch    = T_w_serv + 5
ASHPB.system_update()

# ----------------------------
# 데이터 정의 (Energy, Entropy, Exergy 각각 exch + Mixing 포함)
# ----------------------------
# 데이터 정의 및 딕셔너리 구성

W2kW = 1/1000
Energy = {
    "components": [
        {"value": 0, "sign": 1, "label": ''},
        {"value": ASHPB.Q_r_ext, "sign": 1, "label": 'Heat transfer\nfrom air to\next unit ref'},
        {"value": 0, "sign": 1, "label": ''},
        {"value": 0, "sign": -1, "label": ''},
        {"value": 0, "sign":  1, "label": ''},
        {"value": ASHPB.E_cmp, "sign": 1, "label": 'Compressor\npower input'},
        {"value": 0, "sign": -1, "label": ' '},
        {"value": 0, "sign": -1, "label": ' '},
        {"value": ASHPB.Q_w_sup_exch, "sign": 1, "label": 'Supply water to\n heat exchanger'},
        {"value": 0, "sign": -1, "label": ' '},
        {"value": ASHPB.Q_w_sup_mix, "sign": 1, "label": 'Supply water to\n mixing valve'},
        {"value": 0, "sign": -1, "label": ' '},
        {"value": ASHPB.Q_w_serv, "sign": 1, "label": 'Served hot water'}
    ],
    "color" : 'tw.lime:',
    "ylabel": 'Energy [kW]',
    "ymax"   : 20,
    "yint"   : 5
}
    
Exergy = {
    "components": [
        {"value": ASHPB.X_fan, "sign": 1, "label": 'Elec input\nto fan'},
        {"value": ASHPB.X_r_ext, "sign": 1, "label": 'Heat transfer\nfrom air to\next unit ref'},
        {"value": ASHPB.X_a_ext_in, "sign": 1, "label": 'Inlet air\nto ext unit'},
        {"value": ASHPB.X_c_ext, "sign": -1, "label": 'Consumption\next unit'},
        {"value": ASHPB.X_a_ext_out, "sign":  -1, "label": 'Exhaust air\nfrom ext unit'},
        {"value": ASHPB.X_cmp, "sign": 1, "label": 'Elec input\nto comp'},
        {"value": ASHPB.X_c_r, "sign": -1, "label": 'Consumption\nin ref loop'},
        {"value": ASHPB.X_r_ext, "sign": -1, "label": 'Ext unit from \nref to air'},
        {"value": ASHPB.X_w_sup_exch, "sign":  1, "label": 'Supply water\nto HX'},
        {"value": ASHPB.X_c_exch, "sign": -1, "label": 'Consumption\nin HX'},
        {"value": ASHPB.X_w_sup_mix, "sign": 1, "label": 'Supply water\nto mixing valve'},
        {"value": ASHPB.X_c_mix, "sign": -1, "label": 'Consumption\nin mixing valve'},
        {"value": ASHPB.X_w_serv, "sign": 1, "label": 'Served\nhot water'},
    ],
    "color" : 'tw.purple:',
    "ylabel":  'Exergy [kW]',
    "ymax"   : 4,
    "yint"   : 1
}

data_list = [Energy, Exergy]

# Figure 구성
nrows = 2
ncols = 1

fig, axs = plt.subplots(nrows, ncols, figsize=(dm.cm2in(20)*2, dm.cm2in(8)*2), dpi=200)

for i, data in enumerate(data_list):
    
    ax = axs[i]
    
    # Components
    components = data["components"]
    value      = np.array([comp["value"] * W2kW for comp in components])
    sign       = np.array([comp["sign"] for comp in components])
    labels     = [comp["label"] for comp in components]
    signed     = value * sign
    color      = data["color"]

    # Data length
    DL = len(value)
    
    # x position (bar position)
    x_pos = np.arange(DL)
    
    # bar width, bar margin
    bw = DL * 0.03  
    bm = DL * 0.04 
    
    # ----- 누적 하단 위치 계산 (Waterfall 구조) -----
    bottoms = np.concatenate([[0], np.cumsum(signed)[:-1]])
    tops    = np.concatenate([bottoms[1:], [value[-1]]])  # 다음 막대의 bottom이 이전 막대의 top

    # 마지막 막대 재정의 (총합 혹은 served 등)
    if DL >= 2:
        last_height = bottoms[-1]
        bottoms[-1] = 0
        signed[-1]  = last_height

    # bar plot
    bars = ax.bar(
        x_pos, signed, width=bw, bottom=bottoms,
        color=[color + '500' if h > 0 else color + '100' for h in signed],
        edgecolor=[color + '800' if h > 0 else color + '500' for h in signed],
        linewidth=0.5
    )
    
    # offset
    offset = data["ymax"] * 0.015
    
    # text
    for j, bar in enumerate(bars):
        h = bar.get_height()
        if h == 0:
            bar.set_edgecolor('none')
            bar.set_facecolor('none')
            continue
        text_y = bottoms[j] + h + offset if h >= 0 else tops[j] - h + offset
        ax.text(bar.get_x() + bw/2, text_y, '' if h == 0 else f'{h:.2f}',
            ha='center', va='bottom', fontsize=dm.fs(-2))

    for j in range(len(value) - 1):
        ax.hlines(y=tops[j], xmin=x_pos[j], xmax=x_pos[j+1],
                  color='tw.stone:500', lw=0.25, linestyle=':', zorder=-1)

    # set ticks
    ax.set_xticks(x_pos)
    ax.set_yticks(np.arange(0, data["ymax"] + data["yint"], data["yint"]))
    
    ax.set_xticklabels(labels, ha='center', fontweight=400, fontsize=dm.fs(-1))
    
    # tick parameters
    ax.tick_params(axis='x', length=0)
    ax.tick_params(axis='y', labelsize = dm.fs(-1))
    
    # set limits
    ax.set_xlim(-bm, (DL-1) + bm)
    ax.set_ylim(0, data["ymax"])
    
    # set labels
    ax.set_ylabel(data["ylabel"], fontsize=dm.fs(0), fontweight=400, color='tw.stone:800')
    ax.yaxis.set_label_coords(-0.03, 0.5)

print('최종 사용 온수 에너지율 대비 열교환기 공급 상수도 열에너지율의 비율:', round(ASHPB.Q_w_sup_exch/ASHPB.Q_w_serv*100, 1),'%')
print('최종 사용 온수 에너지율 대비 믹싱 밸브 공급 상수도 열에너지율의 비율:', round(ASHPB.Q_w_sup_mix/ASHPB.Q_w_serv*100, 1),'%')

print('최종 사용 온수 엑서지율 대비 열교환기 공급 상수도 열에너지율의 비율:',  round(ASHPB.X_w_sup_exch/ASHPB.X_w_serv*100, 1),'%')
print('최종 사용 온수 엑서지율 대비 믹싱 밸브 공급 상수도 열에너지율의 비율:', round(ASHPB.X_w_sup_mix/ASHPB.X_w_serv*100, 1),'%')

# 전체 레이아웃 및 저장
plt.subplots_adjust(hspace=0.6)
dm.simple_layout(fig, bbox=[0.01, 0.99, 0.01, 1.0], margins=[0.0, 0.1, 0.05, 0.05])
plt.savefig('figure/ASHPB_waterfall_energy_eff_max.svg', transparent=True)
plt.savefig('figure/ASHPB_waterfall_energy_eff_max.png',dpi=600)
dm.util.save_and_show(fig)

최종 사용 온수 에너지율 대비 열교환기 공급 상수도 열에너지율의 비율: 11.9 %
최종 사용 온수 에너지율 대비 믹싱 밸브 공급 상수도 열에너지율의 비율: 2.1 %
최종 사용 온수 엑서지율 대비 열교환기 공급 상수도 열에너지율의 비율: 1.8 %
최종 사용 온수 엑서지율 대비 믹싱 밸브 공급 상수도 열에너지율의 비율: 0.3 %


In [ ]:
T0 = T0_range[np.argmax(Data['whole']['Ex_eff'])]  # T0 at max exergy efficiency
ASHPB = enex.HeatPumpBoiler_without_tank()
ASHPB.dV_w_serv = 7.5
ASHPB.Q_r_max = 20000

ASHPB.T0          = T0
ASHPB.T_w_serv    = T_w_serv
ASHPB.T_w_sup     = 0.2 * T0 + 10
ASHPB.T_a_ext_out = T0 - 5
ASHPB.T_r_ext     = T0 - 10

ASHPB.T_r_exch    = T_w_serv + 10
ASHPB.T_w_exch    = T_w_serv + 5
ASHPB.system_update()

# ----------------------------
# 데이터 정의 (Energy, Entropy, Exergy 각각 exch + Mixing 포함)
# ----------------------------
# 데이터 정의 및 딕셔너리 구성

W2kW = 1/1000
Energy = {
    "components": [
        {"value": 0, "sign": 1, "label": ''},
        {"value": ASHPB.Q_r_ext, "sign": 1, "label": 'Heat transfer\nfrom air to\next unit ref'},
        {"value": 0, "sign": 1, "label": ''},
        {"value": 0, "sign": -1, "label": ''},
        {"value": 0, "sign":  1, "label": ''},
        {"value": ASHPB.E_cmp, "sign": 1, "label": 'Compressor\npower input'},
        {"value": 0, "sign": -1, "label": ' '},
        {"value": 0, "sign": -1, "label": ' '},
        {"value": ASHPB.Q_w_sup_exch, "sign": 1, "label": 'Supply water to\n heat exchanger'},
        {"value": 0, "sign": -1, "label": ' '},
        {"value": ASHPB.Q_w_sup_mix, "sign": 1, "label": 'Supply water to\n mixing valve'},
        {"value": 0, "sign": -1, "label": ' '},
        {"value": ASHPB.Q_w_serv, "sign": 1, "label": 'Served hot water'}
    ],
    "color" : 'tw.lime:',
    "ylabel": 'Energy [kW]',
    "ymax"   : 24,
    "yint"   : 6
}
    
Exergy = {
    "components": [
        {"value": ASHPB.X_fan, "sign": 1, "label": 'Elec input\nto fan'},
        {"value": ASHPB.X_r_ext, "sign": 1, "label": 'Heat transfer\nfrom air to\next unit ref'},
        {"value": ASHPB.X_a_ext_in, "sign": 1, "label": 'Inlet air\nto ext unit'},
        {"value": ASHPB.X_c_ext, "sign": -1, "label": 'Consumption\next unit'},
        {"value": ASHPB.X_a_ext_out, "sign":  -1, "label": 'Exhaust air\nfrom ext unit'},
        {"value": ASHPB.X_cmp, "sign": 1, "label": 'Elec input\nto comp'},
        {"value": ASHPB.X_c_r, "sign": -1, "label": 'Consumption\nin ref loop'},
        {"value": ASHPB.X_r_ext, "sign": -1, "label": 'Ext unit from \nref to air'},
        {"value": ASHPB.X_w_sup_exch, "sign":  1, "label": 'Supply water\nto HX'},
        {"value": ASHPB.X_c_exch, "sign": -1, "label": 'Consumption\nin HX'},
        {"value": ASHPB.X_w_sup_mix, "sign": 1, "label": 'Supply water\nto mixing valve'},
        {"value": ASHPB.X_c_mix, "sign": -1, "label": 'Consumption\nin mixing valve'},
        {"value": ASHPB.X_w_serv, "sign": 1, "label": 'Served\nhot water'},
    ],
    "color" : 'tw.purple:',
    "ylabel":  'Exergy [kW]',
    "ymax"   : 6,
    "yint"   : 1
}

data_list = [Energy, Exergy]

# Figure 구성
nrows = 2
ncols = 1

fig, axs = plt.subplots(nrows, ncols, figsize=(dm.cm2in(20)*2, dm.cm2in(8)*2), dpi=200)

for i, data in enumerate(data_list):
    
    ax = axs[i]
    
    # Components
    components = data["components"]
    value      = np.array([comp["value"] * W2kW for comp in components])
    sign       = np.array([comp["sign"] for comp in components])
    labels     = [comp["label"] for comp in components]
    signed     = value * sign
    color      = data["color"]

    # Data length
    DL = len(value)
    
    # x position (bar position)
    x_pos = np.arange(DL)
    
    # bar width, bar margin
    bw = DL * 0.03  
    bm = DL * 0.04 
    
    # ----- 누적 하단 위치 계산 (Waterfall 구조) -----
    bottoms = np.concatenate([[0], np.cumsum(signed)[:-1]])
    tops    = np.concatenate([bottoms[1:], [value[-1]]])  # 다음 막대의 bottom이 이전 막대의 top

    # 마지막 막대 재정의 (총합 혹은 served 등)
    if DL >= 2:
        last_height = bottoms[-1]
        bottoms[-1] = 0
        signed[-1]  = last_height

    # bar plot
    bars = ax.bar(
        x_pos, signed, width=bw, bottom=bottoms,
        color=[color + '500' if h > 0 else color + '100' for h in signed],
        edgecolor=[color + '800' if h > 0 else color + '500' for h in signed],
        linewidth=0.5
    )
    
    # offset
    offset = data["ymax"] * 0.015
    
    # text
    for j, bar in enumerate(bars):
        h = bar.get_height()
        if h == 0:
            bar.set_edgecolor('none')
            bar.set_facecolor('none')
            continue
        text_y = bottoms[j] + h + offset if h >= 0 else tops[j] - h + offset
        ax.text(bar.get_x() + bw/2, text_y, '' if h == 0 else f'{h:.2f}',
            ha='center', va='bottom', fontsize=dm.fs(-2))

    for j in range(len(value) - 1):
        ax.hlines(y=tops[j], xmin=x_pos[j], xmax=x_pos[j+1],
                  color='tw.stone:500', lw=0.25, linestyle=':', zorder=-1)

    # set ticks
    ax.set_xticks(x_pos)
    ax.set_yticks(np.arange(0, data["ymax"] + data["yint"], data["yint"]))
    
    ax.set_xticklabels(labels, ha='center', fontweight=400, fontsize=dm.fs(-1))
    
    # tick parameters
    ax.tick_params(axis='x', length=0)
    ax.tick_params(axis='y', labelsize = dm.fs(-1))
    
    # set limits
    ax.set_xlim(-bm, (DL-1) + bm)
    ax.set_ylim(0, data["ymax"])
    
    # set labels
    ax.set_ylabel(data["ylabel"], fontsize=dm.fs(0), fontweight=400, color='tw.stone:800')
    ax.yaxis.set_label_coords(-0.03, 0.5)

print('최종 사용 온수 에너지율 대비 열교환기 공급 상수도 열에너지율의 비율:', round(ASHPB.Q_w_sup_exch/ASHPB.Q_w_serv*100, 1),'%')
print('최종 사용 온수 에너지율 대비 믹싱 밸브 공급 상수도 열에너지율의 비율:', round(ASHPB.Q_w_sup_mix/ASHPB.Q_w_serv*100, 1),'%')

print('최종 사용 온수 엑서지율 대비 열교환기 공급 상수도 열에너지율의 비율:',  round(ASHPB.X_w_sup_exch/ASHPB.X_w_serv*100, 1),'%')
print('최종 사용 온수 엑서지율 대비 믹싱 밸브 공급 상수도 열에너지율의 비율:', round(ASHPB.X_w_sup_mix/ASHPB.X_w_serv*100, 1),'%')

# 전체 레이아웃 및 저장
plt.subplots_adjust(hspace=0.6)
dm.simple_layout(fig, bbox=[0.01, 0.99, 0.01, 1.0], margins=[0.0, 0.1, 0.05, 0.05])
plt.savefig('figure/ASHPB_waterfall_exergy_eff_max.svg', transparent=True)
plt.savefig('figure/ASHPB_waterfall_exergy_eff_max.png',dpi=600)
dm.util.save_and_show(fig)

최종 사용 온수 에너지율 대비 열교환기 공급 상수도 열에너지율의 비율: 25.7 %
최종 사용 온수 에너지율 대비 믹싱 밸브 공급 상수도 열에너지율의 비율: 4.2 %
최종 사용 온수 엑서지율 대비 열교환기 공급 상수도 열에너지율의 비율: 8.3 %
최종 사용 온수 엑서지율 대비 믹싱 밸브 공급 상수도 열에너지율의 비율: 1.3 %
